In [35]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("gyr66/RoBERTa-ext-large-crf-chinese-finetuned-ner", model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained("gyr66/RoBERTa-ext-large-crf-chinese-finetuned-ner")
pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, trust_remote_code=True, aggregation_strategy='simple', device=2)

tokenizer_config.json: 100%|██████████| 1.25k/1.25k [00:00<00:00, 2.38MB/s]
vocab.txt: 100%|██████████| 110k/110k [00:00<00:00, 617kB/s]
tokenizer.json: 100%|██████████| 439k/439k [00:00<00:00, 1.64MB/s]
special_tokens_map.json: 100%|██████████| 695/695 [00:00<00:00, 1.58MB/s]
config.json: 100%|██████████| 2.24k/2.24k [00:00<00:00, 5.35MB/s]


In [38]:
pipe("我叫郭怡然，我在中国科学院大学读书。", aggregation_strategy='simple')

[{'entity_group': 'name',
  'score': 0.088490665,
  'word': '郭 怡 然',
  'start': 2,
  'end': 5},
 {'entity_group': 'organization',
  'score': 0.088490665,
  'word': '中 国 科 学 院',
  'start': 8,
  'end': 13}]

In [39]:
def transform_output(entities, _id):
    result = []
    for entity in entities:
        category = entity['entity_group']
        word = entity['word'].replace(' ', '')
        start = entity['start']
        end = entity['end'] - 1
        result.append({
            'ID': _id,
            'Category': category,
            'Pos_b': start, 
            'Pos_e': end,
            'Privacy': word
        })
    return result

In [40]:
import pandas as pd
import os

In [41]:
text_list = []
id_list = []
res = []
for filename in sorted(os.listdir('data/original/test/text'), key=lambda x: int(x.split('.')[0])):
    with open(os.path.join('data/original/test/text', filename), 'r') as f:
        text = f.read()
    id_list.append(filename.split('.')[0])
    text_list.append(text)
output_list = pipe(text_list, batch_size=128)
for _id, output in zip(id_list, output_list):
    output = transform_output(output, _id)
    res.extend(output)
df = pd.DataFrame(res)
df.to_csv('data/original/test/predict.csv', encoding='utf-8', index=False)

In [42]:
df.head(5)

,ID,Category,Pos_b,Pos_e,Privacy
0,0,book,0,10,《别告诉我你懂ppt》
1,0,book,11,23,《不懂项目管理还敢拼职场》
2,0,book,24,31,《让营销更性感》
3,0,position,33,34,作者
4,0,name,35,36,李治
